## Introducción al Sistema RAG para Discursos del Presidente Javier Milei

Este cuaderno de Google Colab presenta un sistema de **Generación Aumentada por Recuperación (RAG)** diseñado específicamente para analizar y consultar los discursos del presidente Javier Milei. Nuestro objetivo es permitir a usuarios interesados en las políticas de Milei consultar sus posturas, propuestas y argumentos a lo largo del tiempo, basándose en un corpus en constante actualización de sus discursos, comunicados y diversas intervenciones públicas.

### ¿Cómo funciona este sistema?

1.  **Recuperación Inteligente:** Utilizamos **ChromaDB** para realizar búsquedas semánticas. Esto significa que el sistema entiende el "significado" de tu pregunta, no solo las palabras clave, para encontrar los fragmentos más relevantes de los discursos.
2.  **Generación de Respuestas:** La generación de respuestas se potencia con el modelo **Gemini** de Google. Gemini toma los fragmentos relevantes encontrados y los utiliza para formular respuestas coherentes, precisas y contextualizadas.

Este enfoque garantiza que las respuestas no solo sean informativas, sino que también estén directamente respaldadas por las fuentes originales de los discursos, minimizando las "alucinaciones" y proporcionando una base sólida para el análisis de las políticas presidenciales.

## Configuración del Entorno

Primero instalamos las herramientas necesarias. LangChain es nuestra "caja de herramientas" principal para RAG.

In [ ]:
# Instalamos las librerías necesarias para nuestro sistema RAG
# LangChain: la librería más popular para construir sistemas RAG de forma simple
# ChromaDB: nuestra base de datos vectorial para guardar los documentos
# Google GenerativeAI: para conectar con Gemini (solo para generación final)
# sentence-transformers: para embeddings locales multilenguaje
!pip install langchain langchain-google-genai langchain-chroma chromadb sentence-transformers -q

print("Todas las librerías instaladas correctamente")
print("IMPORTANTE: Solo se usará API de Gemini para generación final de respuestas")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.5 MB/s eta 0:00:

In [ ]:
# Importamos todas las herramientas que vamos a necesitar
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions

# Configuración para mostrar mejor los resultados
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")
print("Usando embeddings locales para reducir uso de API")

Librerías cargadas exitosamente
Usando embeddings locales para reducir uso de API


## Configuración de la API de Gemini

Necesitamos configurar nuestra conexión con Gemini de Google. Este será el "cerebro" que generará las respuestas finales.

In [ ]:
# Detectamos si estamos en Google Colab o en un entorno local
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
    print("🔍 Entorno detectado: Google Colab")
except ImportError:
    IN_COLAB = False
    print("🔍 Entorno detectado: Local")

# Obtenemos la clave API según el entorno
if IN_COLAB:
    # En Colab: usar los secretos de Colab (más seguro)
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        print("✅ Clave API cargada desde secretos de Colab")
    except Exception as e:
        print("❌ No se encontró GOOGLE_API_KEY en los secretos de Colab")
        print("   Ve a la barra lateral izquierda > 🔑 Secretos > Agregar GOOGLE_API_KEY")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
else:
    # En local: usar variable de entorno
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("❌ No se encontró GOOGLE_API_KEY en las variables de entorno")
        print("   Opción 1: Agrega GOOGLE_API_KEY a tu archivo .env")
        print("   Opción 2: Ejecuta: export GOOGLE_API_KEY=tu_clave_aqui")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
    else:
        print("✅ Clave API cargada desde variables de entorno")

# Configuramos la variable de entorno para que LangChain la use
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("🚀 Configuración de Gemini completada")

🔍 Entorno detectado: Google Colab
✅ Clave API cargada desde secretos de Colab
🚀 Configuración de Gemini completada


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive montado exitosamente.")
print("Ahora se pueden acceder a los archivos de Drive desde '/content/drive/MyDrive'.")

Mounted at /content/drive
Google Drive montado exitosamente.
Ahora se pueden acceder a los archivos de Drive desde '/content/drive/MyDrive'.


In [ ]:
# @title
import os
import glob
from datetime import datetime
import re
import pandas as pd

# --- Configuración para la carga de archivos locales desde Drive ---
DRIVE_PATH = '/content/drive/MyDrive/nlp/txt para rag'

def limpiar_texto_simple(texto):
    """
    Limpia y preprocesa el texto extraído de archivos locales.
    """
    if not texto:
        return ""

    # 1. Eliminar espacios múltiples y normalizar saltos de línea
    texto = re.sub(r'\n+', '\n', texto)
    texto = re.sub(r' +', ' ', texto)

    # 2. Eliminar caracteres especiales no deseados (mantener puntuación básica y acentos)
    # Esto es una simplificación, ya que el usuario mencionó 'texto plano'.
    # Si se necesita una limpieza más profunda, se puede expandir.
    texto = re.sub(r'[^\w\s.,;:!?¿¡()\-áéíóúÁÉÍÓÚñÑ@]', '', texto)

    # 3. Normalizar saltos de línea y espacios
    lineas = [linea.strip() for linea in texto.split('\n') if linea.strip()]
    texto_limpio = '\n'.join(lineas)

    # 4. Eliminar líneas muy cortas que puedan ser ruido
    lineas_filtradas = []
    for linea in texto_limpio.split('\n'):
        linea_limpia = linea.strip()
        if (len(linea_limpia) > 10 or
            any(caracter in linea_limpia for caracter in ['.', '!', '?', ':', ';']) and len(linea_limpia) > 5):
            lineas_filtradas.append(linea_limpia)

    return '\n'.join(lineas_filtradas)

# --- Lógica principal para cargar archivos de Drive ---
print(f"🚀 INICIANDO CARGA DE ARCHIVOS DE TEXTO DESDE GOOGLE DRIVE")
print("="*60)

discursos_procesados = []

if not os.path.exists(DRIVE_PATH):
    print(f"❌ La carpeta '{DRIVE_PATH}' no se encontró en Google Drive.")
    print("   Asegúrate de que la ruta sea correcta y que Google Drive esté montado.")
else:
    archivos_txt = glob.glob(os.path.join(DRIVE_PATH, '*.txt'))

    if not archivos_txt:
        print(f"⚠️ No se encontraron archivos .txt en '{DRIVE_PATH}'.")
    else:
        print(f"🔍 Encontrados {len(archivos_txt)} archivos .txt en '{DRIVE_PATH}'")
        for i, archivo_path in enumerate(archivos_txt, 1):
            nombre_archivo = os.path.basename(archivo_path)
            titulo = os.path.splitext(nombre_archivo)[0] # Título basado en el nombre del archivo
            fecha_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            print(f"  [{i}/{len(archivos_txt)}] 📄 Procesando: {nombre_archivo}...")
            try:
                with open(archivo_path, 'r', encoding='utf-8') as f:
                    contenido_crudo = f.read()

                contenido_limpio = limpiar_texto_simple(contenido_crudo)

                if len(contenido_limpio) > 100: # Asegurarse de que el contenido no esté vacío
                    discurso_procesado = {
                        'titulo': titulo,
                        'url': archivo_path,
                        'fecha_extraccion': fecha_actual,
                        'contenido_crudo': contenido_crudo,
                        'fecha_publicacion': fecha_actual, # Usar fecha de extracción como placeholder
                        'contenido_limpio': contenido_limpio,
                        'longitud_texto': len(contenido_limpio),
                        'num_parrafos': contenido_limpio.count('\n') + 1,
                        'palabras_aprox': len(contenido_limpio.split())
                    }
                    discursos_procesados.append(discurso_procesado)
                    print(f"     ✅ Cargado: {len(contenido_limpio)} caracteres, {discurso_procesado['palabras_aprox']} palabras aprox.")
                else:
                    print(f"     ⚠️  Descartado: contenido muy corto en {nombre_archivo} después de limpieza")

            except Exception as e:
                print(f"     ❌ Error al leer {nombre_archivo}: {e}")

print(f"\n🎉 PROCESAMIENTO COMPLETADO!")
print(f"✅ Se cargaron y procesaron {len(discursos_procesados)} documentos.")

if discursos_procesados:
    # Mostrar resumen (adaptado para documentos locales)
    print("\n" + "="*80)
    print("📊 RESUMEN DE DOCUMENTOS CARGADOS")
    print("="*80)

    for i, discurso in enumerate(discursos_procesados, 1):
        print(f"\n🎯 DOCUMENTO {i}:")
        print(f"   📖 Título: {discurso['titulo']}")
        print(f"   📅 Fecha de carga: {discurso['fecha_publicacion']}")
        print(f"   📊 Estadísticas:")
        print(f"      - Caracteres: {discurso['longitud_texto']:,}")
        print(f"      - Palabras aprox.: {discurso['palabras_aprox']:,}")
        print(f"      - Párrafos: {discurso['num_parrafos']}")
        print(f"   🔗 Ruta: {discurso['url'][:100]}..." if len(discurso['url']) > 100 else f"   🔗 Ruta: {discurso['url']}")

    # Estadísticas finales
    total_caracteres = sum(d['longitud_texto'] for d in discursos_procesados)
    total_palabras = sum(d['palabras_aprox'] for d in discursos_procesados)

    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"   📚 Total de documentos: {len(discursos_procesados)}")
    print(f"   🔤 Total de caracteres: {total_caracteres:,}")
    print(f"   🗣️  Total de palabras aprox.: {total_palabras:,}")

else:
    print("❌ No se cargaron documentos para procesar.")


🚀 INICIANDO CARGA DE ARCHIVOS DE TEXTO DESDE GOOGLE DRIVE
🔍 Encontrados 50 archivos .txt en '/content/drive/MyDrive/nlp/txt para rag'
  [1/50] 📄 Procesando: audiosjaviermilei127462127479milei-freecubanpodcasts-ivoox92479733.txt...
     ✅ Cargado: 5270 caracteres, 931 palabras aprox.
  [2/50] 📄 Procesando: audiosjaviermileienfrentamientojavie-freecubangonzalezroman-ivoox74747583.txt...
     ✅ Cargado: 7979 caracteres, 1463 palabras aprox.
  [3/50] 📄 Procesando: audiosjaviermileipingpongconjavier-freecubangonzalezroman-ivoox75249217.txt...
     ✅ Cargado: 6572 caracteres, 1180 palabras aprox.
  [4/50] 📄 Procesando: 20251119_021705_216697_Casa_RosadaPresidencia_de_la_Nación.txt...
     ✅ Cargado: 24097 caracteres, 4050 palabras aprox.
  [5/50] 📄 Procesando: 20251119_021705_225632_Casa_RosadaPresidencia_de_la_Nación.txt...
     ✅ Cargado: 8494 caracteres, 1395 palabras aprox.
  [6/50] 📄 Procesando: 20251119_021705_235544_Casa_RosadaPresidencia_de_la_Nación.txt...
     ✅ Cargado: 26516 c

## Dividir los Documentos en Fragmentos

Los documentos largos necesitan dividirse en pedazos más pequeños para que el sistema pueda encontrar información específica. Es como hacer un índice detallado de un libro.

In [ ]:
# El "Text Splitter" es como un bibliotecario que divide documentos grandes
# en secciones manejables, manteniendo el contexto

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cada fragmento tendrá máximo 500 caracteres
    chunk_overlap=50,      # 50 caracteres se superponen entre fragmentos para mantener contexto
    separators=["\n\n", "\n", ".", " "]  # Divide preferentemente por párrafos, luego oraciones
)

# Convertimos nuestros documentos al formato que entiende LangChain
documentos_langchain = []

# Ahora usamos los discursos procesados del web scraping
for doc in discursos_procesados:
    # Cada documento se convierte en un objeto "Document" con contenido y metadata
    documento = Document(
        page_content=doc["contenido_limpio"],
        metadata={
            "titulo": doc["titulo"],
            "fecha_publicacion": doc["fecha_publicacion"],
            "url": doc["url"]
        }
    )
    documentos_langchain.append(documento)

# Dividimos todos los documentos en fragmentos más pequeños
fragmentos = text_splitter.split_documents(documentos_langchain)

print(f"📝 Discursos procesados: {len(discursos_procesados)}")
print(f"🔪 Fragmentos creados: {len(fragmentos)}")
print(f"\n📋 Ejemplo de fragmento:")
print(f"Título: {fragmentos[0].metadata['titulo']}")
print(f"Fecha: {fragmentos[0].metadata['fecha_publicacion']}")
print(f"Contenido: {fragmentos[0].page_content[:200]}...")

📝 Discursos procesados: 50
🔪 Fragmentos creados: 2777

📋 Ejemplo de fragmento:
Título: audiosjaviermilei127462127479milei-freecubanpodcasts-ivoox92479733
Fecha: 2025-11-19 05:01:53
Contenido: Tierra Palabra, el diputado Javier Gerardo Milay. Muchas gracias, señor presidente. En primer lugar, cuando uno mira los considerando de este consenso fiscal, arranca con un eoxímero porque dice que v...


In [ ]:
print(f"Título: {fragmentos[2].metadata['titulo']}")
print(f"Contenido: {fragmentos[2].page_content[:200]}...")

Título: audiosjaviermilei127462127479milei-freecubanpodcasts-ivoox92479733
Contenido: . Es más, ya aún saliendo de una estructura intertemporal que se piensa en estos temas, ni siquiera los demente de los queinesianos proponen que se crezca subiendo impuestos. Por otra parte, dentro de...


## Crear la Base de Conocimiento Vectorial

Ahora convertimos nuestros fragmentos de texto en "vectores" (listas de números) que el sistema puede comparar y buscar de forma inteligente.

In [ ]:
# Los "embeddings" convierten texto en vectores numéricos que representan el significado
# Usamos un modelo local multilenguaje que funciona excelente con español técnico
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"  # Modelo multilenguaje optimizado para español
)

print("Modelo de embeddings local configurado (multilingual-e5-large)")
print("Ventaja: No consume cuota de API, solo procesamiento local")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Modelo de embeddings local configurado (multilingual-e5-large)
Ventaja: No consume cuota de API, solo procesamiento local


In [ ]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto

In [ ]:
# ChromaDB será nuestra "biblioteca inteligente" donde guardamos los vectores
# Es como un bibliotecario que puede encontrar libros por su tema, no solo por título
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

vectorstore = Chroma.from_documents(
    documents=fragmentos,           # Los fragmentos de nuestros documentos
    embedding=embeddings,           # El modelo que convierte texto en vectores
    collection_name="documentos_empresa",  # Nombre de nuestra colección
    persist_directory="./chroma_db"  # Donde se guardan los datos (opcional)
)

print(f"Base de conocimiento vectorial creada con {len(fragmentos)} fragmentos")
print("El sistema ya puede buscar información por significado, no solo por palabras exactas")
print("Los embeddings se procesan localmente sin consumir cuota de Gemini")

Base de conocimiento vectorial creada con 2777 fragmentos
El sistema ya puede buscar información por significado, no solo por palabras exactas
Los embeddings se procesan localmente sin consumir cuota de Gemini


In [ ]:
# Configuramos el modelo Gemini que generará las respuestas finales
# NOTA: Solo este componente consume cuota de API, los embeddings son locales
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",    # Modelo rápido y eficiente de Gemini
    temperature=0.1,             # Baja creatividad = respuestas más precisas y consistentes
    google_api_key=GOOGLE_API_KEY
)

print("Modelo Gemini configurado")
print("   Modelo: gemini-2.5-flash (rápido y preciso)")
print("   Temperatura: 0.1 (respuestas consistentes y factuales)")
print("   IMPORTANTE: Solo la generación final usa API de Gemini")

Modelo Gemini configurado
   Modelo: gemini-2.5-flash (rápido y preciso)
   Temperatura: 0.1 (respuestas consistentes y factuales)
   IMPORTANTE: Solo la generación final usa API de Gemini


## Template de Respuesta

Definimos cómo queremos que Gemini estructure sus respuestas. Es como darle instrucciones específicas sobre su rol y forma de responder.

In [ ]:
# El prompt template es como las instrucciones que le damos a un asistente
# Le decimos exactamente como debe comportarse y que formato usar

# Template mejorado con mas estructura
template_mejorado = """
    Sos un asistente especializado en discursos y políticas del presidente Javier Milei.
    Tu misión es proporcionar información precisa y útil basada ÚNICAMENTE en
    los discursos oficiales disponibles.

    INSTRUCCIONES IMPORTANTES:
    1. Solo usá información que aparece EXPLÍCITAMENTE en los discursos
    2. Si no encontrás la información específica, indicá claramente "No se menciona en los discursos disponibles"
    3. Sé preciso con nombres, fechas, cifras y datos específicos
    4. Usá un tono amigable pero informado y profesional

    FORMATO DE RESPUESTA:
    - Comenzá con un resumen directo de lo encontrado
    - Proporcioná detalles específicos (nombres, fechas, eventos, cifras)
    - Si hay múltiples puntos, numerálos claramente
    - Mencioná en qué discurso y fecha se encuentra cada información
    - Si algo no está claro o no aparece en los documentos, decilo explícitamente

    DISCURSOS DISPONIBLES:
    {context}

    CONSULTA:
    {question}

    RESPUESTA ESTRUCTURADA:
    """

# Creamos el prompt personalizado usando nuestro template
prompt_mejorado = PromptTemplate(
    template=template_mejorado,
    input_variables=["context", "question"]
)

print("Template de respuesta configurado")
print("El asistente seguira instrucciones especificas para dar respuestas precisas")

Template de respuesta configurado
El asistente seguira instrucciones especificas para dar respuestas precisas


## Sistema RAG Completo Opcion A


    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 6, "fetch_k": 15}  # Más documentos para consultas complejas
    )


In [ ]:
# @title
# Función para manejar consultas complejas que requieren múltiples documentos
def consulta_compleja(pregunta):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    """
    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"📋 Pregunta: {pregunta}")
    print("\n" + "="*80)

    # Configuramos para buscar más documentos
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 6, "fetch_k": 15}  # Más documentos para consultas complejas
    )

    # Sistema temporal con más contexto
    sistema_complejo = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_complejo,
        chain_type_kwargs={"prompt": prompt_mejorado},
        return_source_documents=True
    )

    resultado = sistema_complejo({"query": pregunta})

    print(f"🤖 RESPUESTA INTEGRAL:")
    print(resultado["result"])

    print(f"\n" + "="*80)
    print(f"📊 ANÁLISIS DE FUENTES CONSULTADAS")
    print("="*80)

    # Análisis detallado de los documentos fuente
    documentos = resultado['source_documents']

    # Estadísticas generales
    documentos_unicos = list(set([doc.metadata['titulo'] for doc in documentos]))
    fechas_unicas = list(set([doc.metadata.get('fecha_publicacion', 'No disponible') for doc in documentos]))

    print(f"📈 ESTADÍSTICAS:")
    print(f"   • Total de fragmentos consultados: {len(documentos)}")
    print(f"   • Documentos únicos: {len(documentos_unicos)}")
    print(f"   • Rango temporal: {min(fechas_unicas) if fechas_unicas else 'N/A'} a {max(fechas_unicas) if fechas_unicas else 'N/A'}")

    print(f"\n📚 DOCUMENTOS CONSULTADOS (por relevancia):")
    print("-" * 80)

    # Agrupar por documento y mostrar fragmentos relevantes
    documentos_agrupados = {}
    for doc in documentos:
        titulo = doc.metadata['titulo']
        if titulo not in documentos_agrupados:
            documentos_agrupados[titulo] = []
        documentos_agrupados[titulo].append(doc)

    for i, (titulo, fragments) in enumerate(documentos_agrupados.items(), 1):
        # Extraer información del documento
        fecha = fragments[0].metadata.get('fecha_publicacion', 'Fecha no disponible')
        url = fragments[0].metadata.get('url', 'URL no disponible')

        print(f"\n🔹 {i}. {titulo}")
        print(f"   📅 Fecha: {fecha}")
        print(f"   🔗 Fragmentos relevantes: {len(fragments)}")
        print(f"   🌐 URL: {url[:80]}..." if len(url) > 80 else f"   🌐 URL: {url}")

        # Mostrar los fragmentos más relevantes (primeros 2)
        print(f"   📖 Fragmentos destacados:")
        for j, fragment in enumerate(fragments[:2], 1):  # Mostrar solo los 2 primeros por documento
            contenido_limpio = fragment.page_content.replace('\n', ' ').strip()
            if len(contenido_limpio) > 150:
                contenido_limpio = contenido_limpio[:147] + "..."
            print(f"      {j}. {contenido_limpio}")

        if len(fragments) > 2:
            print(f"      ... y {len(fragments) - 2} fragmentos más")

    # Análisis de cobertura temática
    print(f"\n🎯 COBERTURA TEMÁTICA:")
    print("-" * 80)

    # Palabras clave de la pregunta
    palabras_clave = pregunta.lower().split()
    palabras_relevantes = [p for p in palabras_clave if len(p) > 3]  # Filtrar palabras muy cortas

    for palabra in palabras_relevantes[:5]:  # Analizar primeras 5 palabras clave
        menciones = 0
        for doc in documentos:
            if palabra in doc.page_content.lower():
                menciones += 1
        if menciones > 0:
            print(f"   • '{palabra}': mencionado en {menciones} fragmentos")

    # Calidad de las fuentes
    print(f"\n📋 RESUMEN EJECUTIVO DE FUENTES:")
    print("-" * 80)
    print(f"   ✅ Se consultaron {len(documentos_unicos)} documentos únicos")
    print(f"   📅 Discursos desde {min(fechas_unicas) if fechas_unicas else 'N/A'}")
    print(f"   🔍 {len(documentos)} fragmentos analizados para la respuesta")
    print(f"   📊 Cobertura temporal: {len(fechas_unicas)} fechas diferentes")

    return resultado

# Función adicional para mostrar solo un resumen rápido de fuentes
def consulta_rapida(pregunta, mostrar_fuentes_detalladas=False):
    """
    Versión rápida para consultas simples
    """
    print(f"\n⚡ CONSULTA RÁPIDA")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*60)

    resultado = sistema_rag_mejorado({"query": pregunta})

    print(f"🤖 RESPUESTA:")
    print(resultado["result"])

    if mostrar_fuentes_detalladas:
        documentos = resultado['source_documents']
        print(f"\n📚 FUENTES ({len(documentos)} fragmentos):")
        for i, doc in enumerate(documentos, 1):
            titulo = doc.metadata['titulo']
            fecha = doc.metadata.get('fecha_publicacion', 'N/A')
            contenido = doc.page_content.replace('\n', ' ')[:100] + "..."
            print(f"   {i}. [{fecha}] {titulo}")
            print(f"      📄 {contenido}")
    else:
        documentos_unicos = list(set([doc.metadata['titulo'] for doc in resultado['source_documents']]))
        print(f"\n📁 Documentos consultados: {len(documentos_unicos)}")
        for i, titulo in enumerate(documentos_unicos, 1):
            print(f"   {i}. {titulo}")

    return resultado

# Probemos con preguntas que necesitan múltiples fuentes
print("🚀 PROBANDO CONSULTA COMPLEJA MEJORADA")
consulta_compleja("¿Qué dice Javier Milei sobre la economía y las reformas estructurales?")

🚀 PROBANDO CONSULTA COMPLEJA MEJORADA

🧩 CONSULTA COMPLEJA
📋 Pregunta: ¿Qué dice Javier Milei sobre la economía y las reformas estructurales?



ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 

## Sistema RAG Completo Opcion B


    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 8, "fetch_k": 25, "lambda_mult": 0.6}  # Más documentos para consultas complejas
    )



In [ ]:
# @title
# Función para manejar consultas complejas que requieren múltiples documentos
def consulta_compleja(pregunta):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    """
    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"📋 Pregunta: {pregunta}")
    print("\n" + "="*80)

    # Configuramos para buscar más documentos
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 8, "fetch_k": 25, "lambda_mult": 0.6}  # Más documentos para consultas complejas
    )

    # Sistema temporal con más contexto
    sistema_complejo = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_complejo,
        chain_type_kwargs={"prompt": prompt_mejorado},
        return_source_documents=True
    )

    resultado = sistema_complejo({"query": pregunta})

    print(f"🤖 RESPUESTA INTEGRAL:")
    print(resultado["result"])

    print(f"\n" + "="*80)
    print(f"📊 ANÁLISIS DE FUENTES CONSULTADAS")
    print("="*80)

    # Análisis detallado de los documentos fuente
    documentos = resultado['source_documents']

    # Estadísticas generales
    documentos_unicos = list(set([doc.metadata['titulo'] for doc in documentos]))
    fechas_unicas = list(set([doc.metadata.get('fecha_publicacion', 'No disponible') for doc in documentos]))

    print(f"📈 ESTADÍSTICAS:")
    print(f"   • Total de fragmentos consultados: {len(documentos)}")
    print(f"   • Documentos únicos: {len(documentos_unicos)}")
    print(f"   • Rango temporal: {min(fechas_unicas) if fechas_unicas else 'N/A'} a {max(fechas_unicas) if fechas_unicas else 'N/A'}")

    print(f"\n📚 DOCUMENTOS CONSULTADOS (por relevancia):")
    print("-" * 80)

    # Agrupar por documento y mostrar fragmentos relevantes
    documentos_agrupados = {}
    for doc in documentos:
        titulo = doc.metadata['titulo']
        if titulo not in documentos_agrupados:
            documentos_agrupados[titulo] = []
        documentos_agrupados[titulo].append(doc)

    for i, (titulo, fragments) in enumerate(documentos_agrupados.items(), 1):
        # Extraer información del documento
        fecha = fragments[0].metadata.get('fecha_publicacion', 'Fecha no disponible')
        url = fragments[0].metadata.get('url', 'URL no disponible')

        print(f"\n🔹 {i}. {titulo}")
        print(f"   📅 Fecha: {fecha}")
        print(f"   🔗 Fragmentos relevantes: {len(fragments)}")
        print(f"   🌐 URL: {url[:80]}..." if len(url) > 80 else f"   🌐 URL: {url}")

        # Mostrar los fragmentos más relevantes (primeros 2)
        print(f"   📖 Fragmentos destacados:")
        for j, fragment in enumerate(fragments[:2], 1):  # Mostrar solo los 2 primeros por documento
            contenido_limpio = fragment.page_content.replace('\n', ' ').strip()
            if len(contenido_limpio) > 150:
                contenido_limpio = contenido_limpio[:147] + "..."
            print(f"      {j}. {contenido_limpio}")

        if len(fragments) > 2:
            print(f"      ... y {len(fragments) - 2} fragmentos más")

    # Análisis de cobertura temática
    print(f"\n🎯 COBERTURA TEMÁTICA:")
    print("-" * 80)

    # Palabras clave de la pregunta
    palabras_clave = pregunta.lower().split()
    palabras_relevantes = [p for p in palabras_clave if len(p) > 3]  # Filtrar palabras muy cortas

    for palabra in palabras_relevantes[:5]:  # Analizar primeras 5 palabras clave
        menciones = 0
        for doc in documentos:
            if palabra in doc.page_content.lower():
                menciones += 1
        if menciones > 0:
            print(f"   • '{palabra}': mencionado en {menciones} fragmentos")

    # Calidad de las fuentes
    print(f"\n📋 RESUMEN EJECUTIVO DE FUENTES:")
    print("-" * 80)
    print(f"   ✅ Se consultaron {len(documentos_unicos)} documentos únicos")
    print(f"   📅 Discursos desde {min(fechas_unicas) if fechas_unicas else 'N/A'}")
    print(f"   🔍 {len(documentos)} fragmentos analizados para la respuesta")
    print(f"   📊 Cobertura temporal: {len(fechas_unicas)} fechas diferentes")

    return resultado

# Función adicional para mostrar solo un resumen rápido de fuentes
def consulta_rapida(pregunta, mostrar_fuentes_detalladas=False):
    """
    Versión rápida para consultas simples
    """
    print(f"\n⚡ CONSULTA RÁPIDA")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*60)

    resultado = sistema_rag_mejorado({"query": pregunta})

    print(f"🤖 RESPUESTA:")
    print(resultado["result"])

    if mostrar_fuentes_detalladas:
        documentos = resultado['source_documents']
        print(f"\n📚 FUENTES ({len(documentos)} fragmentos):")
        for i, doc in enumerate(documentos, 1):
            titulo = doc.metadata['titulo']
            fecha = doc.metadata.get('fecha_publicacion', 'N/A')
            contenido = doc.page_content.replace('\n', ' ')[:100] + "..."
            print(f"   {i}. [{fecha}] {titulo}")
            print(f"      📄 {contenido}")
    else:
        documentos_unicos = list(set([doc.metadata['titulo'] for doc in resultado['source_documents']]))
        print(f"\n📁 Documentos consultados: {len(documentos_unicos)}")
        for i, titulo in enumerate(documentos_unicos, 1):
            print(f"   {i}. {titulo}")

    return resultado

# Probemos con preguntas que necesitan múltiples fuentes
print("🚀 PROBANDO CONSULTA COMPLEJA MEJORADA")
consulta_compleja("¿Qué dice Javier Milei sobre la economía y las reformas estructurales?")

🚀 PROBANDO CONSULTA COMPLEJA MEJORADA

🧩 CONSULTA COMPLEJA
📋 Pregunta: ¿Qué dice Javier Milei sobre la economía y las reformas estructurales?

🤖 RESPUESTA INTEGRAL:
Aquí tienes la información sobre la economía y las reformas estructurales, basada en los discursos disponibles del Presidente Javier Milei:

El Presidente Javier Milei enfatiza que el ordenamiento de la macroeconomía es la base del crecimiento económico y anuncia una serie de reformas económicas para impulsar un crecimiento inédito y dinamizar la microeconomía. Destaca el equilibrio fiscal como un fundamento clave para el crecimiento sistemático y propone una secuencia específica para las reformas tributarias, laborales y la apertura comercial, buscando evitar costos sociales negativos.

A continuación, los detalles específicos:

1.  **Ordenamiento Macroeconómico y Crecimiento:**
    *   Durante el último año y medio, se dedicaron a "ordenar la macro a toda costa" porque es la "base del crecimiento económico".
    *   Se an

{'query': '¿Qué dice Javier Milei sobre la economía y las reformas estructurales?',
 'result': 'Aquí tienes la información sobre la economía y las reformas estructurales, basada en los discursos disponibles del Presidente Javier Milei:\n\nEl Presidente Javier Milei enfatiza que el ordenamiento de la macroeconomía es la base del crecimiento económico y anuncia una serie de reformas económicas para impulsar un crecimiento inédito y dinamizar la microeconomía. Destaca el equilibrio fiscal como un fundamento clave para el crecimiento sistemático y propone una secuencia específica para las reformas tributarias, laborales y la apertura comercial, buscando evitar costos sociales negativos.\n\nA continuación, los detalles específicos:\n\n1.  **Ordenamiento Macroeconómico y Crecimiento:**\n    *   Durante el último año y medio, se dedicaron a "ordenar la macro a toda costa" porque es la "base del crecimiento económico".\n    *   Se anuncian "una serie de reformas económicas dirigidas a apuntala

## Sistema RAG Completo Opcion C


    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": max_documentos,
            "fetch_k": 25,  # Optimizado
            "lambda_mult": 0.6  # Mejor balance diversidad/relevancia
        }
    )

In [ ]:
# @title
# Función optimizada para consultas complejas
def consulta_compleja(pregunta, max_documentos=6, max_fragmentos_por_doc=2):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    Optimizada para rendimiento y claridad
    """
    # Timer para medir performance
    import time
    inicio = time.time()

    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"📋 Pregunta: {pregunta}")
    print("=" * 80)

    # Configuración optimizada del retriever
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": max_documentos,
            "fetch_k": 25,  # Optimizado
            "lambda_mult": 0.6  # Mejor balance diversidad/relevancia
        }
    )

    # Reutilizar sistema existente si es posible, sino crear uno temporal
    try:
        sistema_complejo = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever_complejo,
            chain_type_kwargs={"prompt": prompt_mejorado},
            return_source_documents=True
        )

        resultado = sistema_complejo.invoke({"query": pregunta})

    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return None

    tiempo_procesamiento = time.time() - inicio

    # --- PRESENTACIÓN DE RESULTADOS OPTIMIZADA ---

    # 1. RESPUESTA PRINCIPAL
    print(f"\n🤖 RESPUESTA INTEGRAL ({tiempo_procesamiento:.1f}s):")
    print("-" * 50)
    print(resultado["result"])

    # 2. ANÁLISIS DE FUENTES MEJORADO
    documentos = resultado['source_documents']
    if not documentos:
        print(f"\n⚠️ No se encontraron documentos relevantes")
        return resultado

    print(f"\n" + "=" * 80)
    print(f"📊 ANÁLISIS DE FUENTES")
    print("=" * 80)

    # Procesamiento más eficiente de documentos
    documentos_agrupados = {}
    todas_fechas = []

    for doc in documentos:
        titulo = doc.metadata['titulo']
        fecha = doc.metadata.get('fecha_publicacion', 'No disponible')

        if titulo not in documentos_agrupados:
            documentos_agrupados[titulo] = {
                'fragments': [],
                'fecha': fecha,
                'url': doc.metadata.get('url', 'URL no disponible')
            }
        documentos_agrupados[titulo]['fragments'].append(doc)
        todas_fechas.append(fecha)

    # Estadísticas rápidas
    fechas_unicas = list(set(todas_fechas))
    fechas_unicas.sort()  # Ordenar cronológicamente

    print(f"📈 ESTADÍSTICAS:")
    print(f"   • Fragmentos consultados: {len(documentos)}")
    print(f"   • Documentos únicos: {len(documentos_agrupados)}")
    print(f"   • Rango temporal: {fechas_unicas[0] if fechas_unicas else 'N/A'} a {fechas_unicas[-1] if fechas_unicas else 'N/A'}")
    print(f"   • Tiempo de respuesta: {tiempo_procesamiento:.2f}s")

    # 3. DOCUMENTOS CONSULTADOS - VISTA MEJORADA
    print(f"\n📚 DOCUMENTOS CONSULTADOS:")
    print("-" * 80)

    for i, (titulo, info) in enumerate(documentos_agrupados.items(), 1):
        fragments = info['fragments']
        fecha = info['fecha']
        url = info['url']

        # Formato compacto y legible
        print(f"\n🔹 {i}. {titulo}")
        print(f"   📅 {fecha} | 🔗 {len(fragments)} fragmentos")

        # URL acortada inteligentemente
        url_display = url if len(url) <= 70 else f"{url[:67]}..."
        print(f"   🌐 {url_display}")

        # Fragmentos más relevantes (optimizado)
        if fragments:
            print(f"   📖 Fragmentos destacados:")
            for j, fragment in enumerate(fragments[:max_fragmentos_por_doc], 1):
                contenido = fragment.page_content.replace('\n', ' ').strip()
                # Longitud adaptativa según contenido
                max_len = 120 if j == 1 else 80
                if len(contenido) > max_len:
                    contenido = contenido[:max_len-3] + "..."
                print(f"      {j}. {contenido}")

            if len(fragments) > max_fragmentos_por_doc:
                print(f"      ... y {len(fragments) - max_fragmentos_por_doc} fragmentos más")

    # 4. ANÁLISIS TEMÁTICO MEJORADO
    print(f"\n🎯 ANÁLISIS TEMÁTICO:")
    print("-" * 80)

    # Palabras clave más inteligentes (excluir stop words simples)
    stop_words = {'qué', 'sobre', 'para', 'como', 'soy', 'del', 'de', 'y', 'en', 'un', 'una'}
    palabras_clave = [p for p in pregunta.lower().split()
                     if len(p) > 3 and p not in stop_words][:6]  # Máximo 6 palabras

    if palabras_clave:
        cobertura = {}
        for palabra in palabras_clave:
            menciones = sum(1 for doc in documentos if palabra in doc.page_content.lower())
            if menciones > 0:
                cobertura[palabra] = menciones

        if cobertura:
            # Ordenar por relevancia (más menciones primero)
            for palabra, count in sorted(cobertura.items(), key=lambda x: x[1], reverse=True):
                porcentaje = (count / len(documentos)) * 100
                print(f"   • '{palabra}': {count} fragmentos ({porcentaje:.0f}% cobertura)")
        else:
            print("   ⚠️  No se encontraron coincidencias directas con palabras clave")
    else:
        print("   ℹ️  Pregunta muy general para análisis temático")

    # 5. RESUMEN EJECUTIVO OPTIMIZADO
    print(f"\n📋 RESUMEN EJECUTIVO:")
    print("-" * 80)
    print(f"   ✅ {len(documentos_agrupados)} documentos únicos consultados")
    print(f"   📅 {len(fechas_unicas)} fechas diferentes cubiertas")
    print(f"   🔍 {len(documentos)} fragmentos analizados")
    print(f"   ⚡ {tiempo_procesamiento:.2f} segundos de procesamiento")
    print(f"   📏 Cobertura temporal: {len(fechas_unicas)} eventos distintos")

    return resultado

# Ejemplo de uso simplificado
if __name__ == "__main__":
    print("🚀 SISTEMA RAG OPTIMIZADO - LISTO")
    print("=" * 50)

    # Probar con una consulta de ejemplo
    consulta_compleja("¿Qué dice Javier Milei sobre la economía y las reformas estructurales?")

🚀 SISTEMA RAG OPTIMIZADO - LISTO

🧩 CONSULTA COMPLEJA
📋 Pregunta: ¿Qué dice Javier Milei sobre la economía y las reformas estructurales?

🤖 RESPUESTA INTEGRAL (8.3s):
--------------------------------------------------
¡Hola! Con gusto te proporciono la información sobre la economía y las reformas estructurales que Javier Milei menciona en el discurso disponible.

En el Discurso del Presidente de la Nación, Javier Milei, en la Bolsa de Comercio de Córdoba, se destaca la importancia de ordenar la macroeconomía como base para el crecimiento y se anuncian reformas económicas para dinamizar la microeconomía. Se enfatiza el equilibrio fiscal como pilar fundamental para el crecimiento sostenido y se rechazan las políticas que llevaron a la pobreza y al borde de la hiperinflación.

Aquí los detalles específicos:

1.  **Ordenamiento Macroeconómico y Crecimiento:**
    *   Se menciona que durante el "último año y medio" (aproximadamente 18 meses) se dedicaron a "ordenar la macro a toda costa", c

## SISTEMA DE CARGA DE ARCHIVOS PARA JUPYTER



In [ ]:
!pip install pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 6.3 MB/s eta 0:00:00


In [ ]:
# @title
import os
import glob
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from langchain.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configurar el text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)

# Widgets para interfaz interactiva
progress = widgets.FloatProgress(value=0.0, min=0, max=1.0, description='Progreso:')
status_text = widgets.HTML(value="<i>Listo para cargar archivos...</i>")
file_stats = widgets.HTML(value="")
result_display = widgets.Output()

def detectar_o_crear_carpeta(ruta_carpeta="./documentos/"):
    """
    Detecta si la carpeta existe, si no, la crea y da instrucciones
    """
    if os.path.exists(ruta_carpeta):
        archivos = glob.glob(os.path.join(ruta_carpeta, "*"))
        archivos_soportados = [f for f in archivos if any(f.endswith(ext) for ext in ['.pdf', '.txt', '.docx'])]

        with result_display:
            if archivos_soportados:
                print(f"✅ Carpeta encontrada: {ruta_carpeta}")
                print(f"📁 Archivos detectados: {len(archivos_soportados)}")
                for archivo in archivos_soportados[:5]:  # Mostrar primeros 5
                    print(f"   📄 {os.path.basename(archivo)}")
                if len(archivos_soportados) > 5:
                    print(f"   ... y {len(archivos_soportados) - 5} más")
            else:
                print(f"📁 Carpeta encontrada pero sin archivos PDF/TXT/DOCX")
                print("💡 Agrega archivos a la carpeta 'documentos/' y vuelve a intentar")

        return True, archivos_soportados
    else:
        # Crear la carpeta
        os.makedirs(ruta_carpeta, exist_ok=True)

        with result_display:
            print(f"📂 Se creó la carpeta: {ruta_carpeta}")
            print("\n📋 INSTRUCCIONES PARA AGREGAR ARCHIVOS:")
            print("1. Ve a la pestaña 'Archivos' en el lateral izquierdo")
            print("2. Busca la carpeta 'documentos'")
            print("3. Arrastra tus archivos PDF, TXT o DOCX allí")
            print("4. Vuelve a hacer clic en 'Cargar carpeta documentos/'")
            print("\n💡 También puedes usar 'Cargar archivos específicos' para rutas exactas")

        return False, []

def cargar_archivos_jupyter(archivos_lista):
    """
    Versión optimizada para Jupyter - carga archivos con barra de progreso
    """
    documentos = []
    total_archivos = len(archivos_lista)

    if total_archivos == 0:
        status_text.value = "<span style='color: orange'>⚠️ No hay archivos para procesar</span>"
        return []

    status_text.value = f"<span style='color: blue'>📁 Procesando {total_archivos} archivos...</span>"

    for i, archivo_path in enumerate(archivos_lista):
        try:
            # Actualizar progreso
            progress.value = (i + 1) / total_archivos
            nombre_archivo = os.path.basename(archivo_path)

            # Mostrar archivo actual
            status_text.value = f"<span style='color: blue'>📖 Leyendo: {nombre_archivo} ({i+1}/{total_archivos})</span>"

            if archivo_path.lower().endswith('.pdf'):
                loader = PyPDFLoader(archivo_path)
                docs = loader.load()
                tipo = "📄 PDF"

            elif archivo_path.lower().endswith('.txt'):
                loader = TextLoader(archivo_path, encoding='utf-8')
                docs = loader.load()
                tipo = "📝 TXT"

            elif archivo_path.lower().endswith('.docx'):
                loader = UnstructuredWordDocumentLoader(archivo_path)
                docs = loader.load()
                tipo = "📋 DOCX"

            else:
                with result_display:
                    print(f"❌ Formato no soportado: {nombre_archivo}")
                continue

            # Agregar metadata
            for doc in docs:
                doc.metadata.update({
                    "fuente": "archivo_local",
                    "nombre_archivo": nombre_archivo,
                    "ruta_archivo": archivo_path,
                    "tipo_archivo": tipo.strip()
                })

            documentos.extend(docs)

            # Mostrar éxito en el output
            with result_display:
                print(f"✅ {tipo}: {nombre_archivo} → {len(docs)} páginas/secciones")

        except Exception as e:
            with result_display:
                print(f"❌ Error en {archivo_path}: {str(e)}")
            continue

    progress.value = 1.0
    return documentos

def procesar_archivos_interactivo(vectorstore, archivos_lista):
    """
    Función interactiva para procesar archivos en Jupyter
    """
    # Limpiar output anterior
    result_display.clear_output()

    with result_display:
        print("🔄 INICIANDO PROCESAMIENTO DE ARCHIVOS")
        print("=" * 50)

    # 1. Cargar archivos
    documentos = cargar_archivos_jupyter(archivos_lista)

    if not documentos:
        status_text.value = "<span style='color: red'>❌ No se pudieron cargar archivos</span>"
        return 0, 0

    # 2. Dividir en fragmentos
    with result_display:
        print(f"\n🔪 Dividiendo {len(documentos)} documentos en fragmentos...")

    fragmentos = text_splitter.split_documents(documentos)

    with result_display:
        print(f"📝 Fragmentos creados: {len(fragmentos)}")

    # 3. Agregar al vectorstore
    with result_display:
        print(f"\n📚 Agregando al vectorstore...")

    try:
        vectorstore.add_documents(fragmentos)

        with result_display:
            print(f"✅ ✅ {len(fragmentos)} fragmentos agregados exitosamente!")

        # 4. Mostrar estadísticas bonitas
        mostrar_estadisticas(documentos, fragmentos)

        status_text.value = "<span style='color: green'>🎉 ¡Carga completada exitosamente!</span>"

    except Exception as e:
        with result_display:
            print(f"❌ Error agregando documentos: {e}")
        status_text.value = "<span style='color: red'>❌ Error en la carga</span>"
        return len(documentos), 0

    return len(documentos), len(fragmentos)

def mostrar_estadisticas(documentos, fragmentos):
    """
    Muestra estadísticas formateadas para Jupyter
    """
    # Contar tipos de archivos
    tipos_archivos = {}
    for doc in documentos:
        archivo = doc.metadata.get('nombre_archivo', '')
        if archivo.endswith('.pdf'):
            tipos_archivos['PDF'] = tipos_archivos.get('PDF', 0) + 1
        elif archivo.endswith('.txt'):
            tipos_archivos['TXT'] = tipos_archivos.get('TXT', 0) + 1
        elif archivo.endswith('.docx'):
            tipos_archivos['DOCX'] = tipos_archivos.get('DOCX', 0) + 1

    with result_display:
        print(f"\n📊 RESUMEN DE CARGA:")
        print("=" * 50)
        print(f"   📄 Documentos originales: {len(documentos)}")
        print(f"   🧩 Fragmentos generados: {len(fragmentos)}")
        print(f"   📈 Ratio fragmentos/doc: {len(fragmentos)/len(documentos):.1f}")

        if tipos_archivos:
            print(f"\n   📁 Tipos de archivos:")
            for tipo, cantidad in tipos_archivos.items():
                print(f"      • {tipo}: {cantidad}")

        print(f"\n   💾 Base de conocimiento actualizada")
        print("=" * 50)

# FUNCIONES DE CONVENIENCIA MEJORADAS
def cargar_desde_carpeta(vectorstore, ruta_carpeta="./documentos/"):
    """
    Carga todos los archivos de una carpeta - CON DETECCIÓN AUTOMÁTICA
    """
    # Detectar o crear carpeta
    carpeta_existe, archivos_lista = detectar_o_crear_carpeta(ruta_carpeta)

    if not carpeta_existe or not archivos_lista:
        return 0, 0

    archivos_lista.sort()  # Ordenar alfabéticamente

    with result_display:
        print(f"📂 Carpeta: {ruta_carpeta}")
        print(f"🔍 Procesando {len(archivos_lista)} archivos...")

    return procesar_archivos_interactivo(vectorstore, archivos_lista)

def cargar_archivos_especificos(vectorstore, lista_rutas):
    """
    Carga una lista específica de archivos - CON VERIFICACIÓN MEJORADA
    """
    # Verificar que los archivos existen
    archivos_validos = []
    archivos_no_encontrados = []

    for ruta in lista_rutas:
        if os.path.exists(ruta):
            archivos_validos.append(ruta)
        else:
            archivos_no_encontrados.append(ruta)

    if archivos_no_encontrados:
        with result_display:
            print("⚠️ Archivos no encontrados:")
            for ruta in archivos_no_encontrados:
                print(f"   ❌ {ruta}")

    if not archivos_validos:
        with result_display:
            print("❌ No hay archivos válidos para cargar")
        return 0, 0

    with result_display:
        print(f"🔍 Procesando {len(archivos_validos)} archivos específicos...")

    return procesar_archivos_interactivo(vectorstore, archivos_validos)

# WIDGETS PARA INTERFAZ VISUAL MEJORADA
def crear_interfaz_carga():
    """
    Crea una interfaz visual mejorada para cargar archivos
    """
    # Botones de acción
    btn_carpeta = widgets.Button(
        description="📂 Cargar carpeta 'documentos/'",
        button_style='primary',
        tooltip='Carga todos los archivos de la carpeta documentos/',
        layout=widgets.Layout(width='300px', height='40px')
    )

    btn_explorar = widgets.Button(
        description="🔍 Explorar carpeta actual",
        button_style='info',
        tooltip='Ver qué archivos hay disponibles',
        layout=widgets.Layout(width='200px', height='40px')
    )

    btn_manual = widgets.Button(
        description="🔧 Cargar archivos específicos",
        button_style='info',
        tooltip='Especificar rutas manualmente',
        layout=widgets.Layout(width='250px', height='40px')
    )

    btn_limpiar = widgets.Button(
        description="🧹 Limpiar resultados",
        button_style='warning',
        layout=widgets.Layout(width='150px', height='40px')
    )

    # Campo para rutas manuales
    texto_rutas = widgets.Textarea(
        value="",
        placeholder="Pega las rutas de archivos, una por línea:\n/documentos/doc1.pdf\n/documentos/doc2.txt\nO usa rutas absolutas: /content/mi_archivo.pdf",
        description="Archivos:",
        disabled=False,
        layout=widgets.Layout(width="90%", height="100px")
    )

    # Función para botón de carpeta
    def on_btn_carpeta_click(b):
        cargar_desde_carpeta(vectorstore)

    # Función para explorar
    def on_btn_explorar_click(b):
        result_display.clear_output()
        detectar_o_crear_carpeta("./documentos/")

    # Función para botón manual
    def on_btn_manual_click(b):
        if texto_rutas.value.strip():
            rutas = [r.strip() for r in texto_rutas.value.split('\n') if r.strip()]
            cargar_archivos_especificos(vectorstore, rutas)
        else:
            with result_display:
                print("⚠️ Ingresa al menos una ruta de archivo")

    # Función para limpiar
    def on_btn_limpiar_click(b):
        result_display.clear_output()
        status_text.value = "<i>Resultados limpiados...</i>"
        progress.value = 0.0

    # Conectar botones
    btn_carpeta.on_click(on_btn_carpeta_click)
    btn_explorar.on_click(on_btn_explorar_click)
    btn_manual.on_click(on_btn_manual_click)
    btn_limpiar.on_click(on_btn_limpiar_click)

    # Diseño de la interfaz
    controles = widgets.VBox([
        widgets.HTML(value="<h3>🎯 Opciones de Carga:</h3>"),
        widgets.HBox([btn_carpeta, btn_explorar]),
        widgets.HBox([btn_manual, btn_limpiar]),
        widgets.HTML(value="<hr>"),
        widgets.HTML(value="<h4>📋 Carga Manual de Archivos:</h4>"),
        texto_rutas
    ])

    return widgets.VBox([
        widgets.HTML(value="<h2>🗂️ Cargador de Archivos para RAG</h2>"),
        widgets.HTML(value="<p>Carga archivos PDF, TXT y DOCX a tu sistema de consultas</p>"),
        controles,
        widgets.HTML(value="<hr>"),
        progress,
        status_text,
        file_stats,
        widgets.HTML(value="<h4>📝 Log de ejecución:</h4>"),
        result_display
    ])

# INICIALIZAR INTERFAZ
def inicializar_cargador():
    """
    Inicializa y muestra el cargador interactivo
    """
    display(crear_interfaz_carga())

    # Mostrar estado inicial
    with result_display:
        print("🔍 Estado inicial del sistema:")
    detectar_o_crear_carpeta("./documentos/")

# USO RÁPIDO (sin interfaz)
def carga_rapida(vectorstore, ruta_archivo):
    """
    Función rápida para cargar un solo archivo
    """
    if os.path.exists(ruta_archivo):
        return procesar_archivos_interactivo(vectorstore, [ruta_archivo])
    else:
        print(f"❌ Archivo no encontrado: {ruta_archivo}")
        return 0, 0

# FUNCIÓN PARA SUBIR ARCHIVOS DIRECTAMENTE EN COLAB
def instrucciones_subida_colab():
    """
    Muestra instrucciones específicas para Google Colab
    """
    with result_display:
        print("🚀 INSTRUCCIONES PARA GOOGLE COLAB:")
        print("=" * 50)
        print("1. Haz clic en el icono de 📁 (Archivos) en el lateral izquierdo")
        print("2. Haz clic derecho en la carpeta 'documentos' o en espacio vacío")
        print("3. Selecciona 'Subir' y elige tus archivos PDF/TXT/DOCX")
        print("4. Espera a que se completen las subidas")
        print("5. Vuelve a esta celda y haz clic en 'Cargar carpeta documentos/'")
        print("\n💡 También puedes usar:")
        print("   • 'Explorar carpeta actual' para ver qué hay")
        print("   • 'Cargar archivos específicos' para rutas exactas")

# EJECUTAR ESTO EN JUPYTER PARA USAR LA INTERFAZ:
print("✅ Sistema de carga listo - Ejecuta: inicializar_cargador()")

✅ Sistema de carga listo - Ejecuta: inicializar_cargador()


In [ ]:
inicializar_cargador()

In [ ]:
consulta_compleja("De que trato el discurso de asuncion de Milei?")


🧩 CONSULTA COMPLEJA
📋 Pregunta: De que trato el discurso de asuncion de Milei?

🤖 RESPUESTA INTEGRAL (3.8s):
--------------------------------------------------
Hola! Con gusto te ayudo con tu consulta.

**Resumen directo:**
La información sobre el contenido del discurso de asunción del Presidente Javier Milei no se encuentra explícitamente detallada en los discursos disponibles.

**Detalles específicos:**
No se menciona en los discursos disponibles. Los documentos proporcionados incluyen:
1.  Discurso en la inauguración de la fábrica de Lamb Weston en Mar del Plata.
2.  Discurso "Plan Argentina Grande otra vez" en Sidersa.
3.  Discurso en la 80 sesión de la Asamblea General de la ONU, Nueva York.
4.  Un fragmento de discurso que menciona la presentación del presupuesto y la visión a 10, 20 y 30 años para Argentina.

Ninguno de estos documentos describe el contenido específico del discurso de asunción.

📊 ANÁLISIS DE FUENTES
📈 ESTADÍSTICAS:
   • Fragmentos consultados: 6
   • Documento

{'query': 'De que trato el discurso de asuncion de Milei?',
 'result': 'Hola! Con gusto te ayudo con tu consulta.\n\n**Resumen directo:**\nLa información sobre el contenido del discurso de asunción del Presidente Javier Milei no se encuentra explícitamente detallada en los discursos disponibles.\n\n**Detalles específicos:**\nNo se menciona en los discursos disponibles. Los documentos proporcionados incluyen:\n1.  Discurso en la inauguración de la fábrica de Lamb Weston en Mar del Plata.\n2.  Discurso "Plan Argentina Grande otra vez" en Sidersa.\n3.  Discurso en la 80 sesión de la Asamblea General de la ONU, Nueva York.\n4.  Un fragmento de discurso que menciona la presentación del presupuesto y la visión a 10, 20 y 30 años para Argentina.\n\nNinguno de estos documentos describe el contenido específico del discurso de asunción.',
 'source_documents': [Document(id='e84d94d7-7509-44c3-a910-4679adc62635', metadata={'titulo': 'Casa RosadaPresidencia de la Nación', 'url': 'https://www.casaro